# ETL
##### Extract, Transform, Load
Get data from API/CSV/Other sources and store them in our own database

### READING FROM CSV

In [ ]:
import pandas as pd

file = "Resources/sample_data.csv"
# import os
# file = os.path.join("Resources", "sample_data.csv")
file_df = pd.read_csv(file)
file_df.head()

# ----------------------------------------------------------------------------------------
# for multiple csv files
year = ['2016', '2017', '2018', '2019']

df_list = {}
for y in year:
    file_name = f"Resources/san_francisco-censustracts-{y}-3-All-HourlyAggregate.csv"
    df_list[y] = pd.read_csv(file_name, usecols=['sourceid', 
                                                 'dstid', 
                                                 'hod', 
                                                 'mean_travel_time', 
                                                 'standard_deviation_travel_time'])
    
for y in year:
    print(df_list[y].head())
    print('------------------------------')

### READING FROM API

### STORE IN SQLITE

In [ ]:
# TAKEN FROM: /10-Advanced-Data-Storage-and-Retrieval/1/Activities/10-Ins_SQL_Alchemy_Revisited/Solved/Alchemy_Annotated.ipynb

# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 

# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()

# Creates Classes which will serve as the anchor points for our Tables
class My_Class(Base):
    __tablename__ = 'my_table'
    id = Column(Integer, primary_key=True)
    name = Column(String(255))
    color = Column(String(255))
    num = Column(Integer)

# Creates a sqlite file and a connection to our DB
engine = create_engine("sqlite:///my_sqlite_file.sqlite")
conn = engine.connect()

# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)

# Use this to clear out the db
# ----------------------------------
# Base.metadata.drop_all(engine)

# Create a Session Object to Connect to DB
# ----------------------------------
# Session is a temporary binding to our DB
from sqlalchemy.orm import Session
session = Session(bind=engine)

# Create Instances of my class
class_instance_1 = My_Class(name='some_name', color='some_color', num=1)
class_instance_1 = My_Class(name="some_other_name", color="some_other_color", num=2)

# Add Records to the Appropriate DB
# ----------------------------------
# Use the SQL ALchemy methods to run simple "INSERT" statements using the classes and objects  
session.add(class_instance_1)
session.add(class_instance_2)
session.commit()

# Query the Tables / Also shown in section READING FROM SQLITE
# ----------------------------------
# Perform a simple query of the database
# my_list_of_rows = session.query(My_Class)
# for row in my_list_of_rows:
#     print(row.name)
#     print(row.color)
#     print(row.age)

### STORE IN SQL

### STORE IN MONGO DB

In [ ]:
# TAKEN FROM: 12-Web-Scraping-and-Document-Databases/1/Activities/05-Ins_PyMongo/Solved/IntroToPymongo.py

# Module used to connect Python with MongoDb
import pymongo

# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define the 'classDB' database in Mongo
db = client.classDB

# Query all students
# Here, db.students refers to the collection 'classroom '
classroom = db.classroom.find()

# Iterate through each student in the collection
for student in classroom:
    print(student)

# Insert a document into the 'students' collection
db.classroom.insert_one(
    {
        'name': 'Ahmed',
        'row': 3,
        'favorite_python_library': 'Matplotlib',
        'hobbies': ['Running', 'Stargazing', 'Reading']
    }
)

# Update a document
db.classroom.update_one(
    {'name': 'Ahmed'},
    {'$set':
        {'row': 4}
     }
)

# Add an item to a document array
db.classroom.update_one(
    {'name': 'Ahmed'},
    {'$push':
        {'hobbies': 'Listening to country music'}
     }
)

# Delete a field from a document
db.classroom.update_one({'name': 'Ahmed'},
                        {'$unset':
                         {'gavecandy': ""}
                         }
                        )


# Delete a document from a collection
db.classroom.delete_one(
    {'name': 'Ahmed'}
)

### READ FROM SQLITE

In [ ]:
# TAKEN FROM: /10-Advanced-Data-Storage-and-Retrieval/1/Activities/04-Stu_ReadAllTheSQLs

from sqlalchemy import create_engine
# Path to sqlite
database_path = "../../my_file.sqlite"

# Create an engine that can talk to the database
engine = create_engine(f"sqlite:///{database_path}")

# Query All Records in the the Database
data = engine.execute("SELECT * FROM my_table")
for row in data:
    # returns a list 
    print(row)
    
# Store in pandas
import pandas as pd
conn = engine.connect()

# Query All Records in the the Database
data_df = pd.read_sql("SELECT * FROM my_table", conn)
print(data_df.row)

# Merging colums template
# combined_data = pd.merge(data_df_1, data_df_2, on="col_name", how="inner")

In [ ]:
# ----------------------------------------------------------------------------------------
# Reading data using classes
# TAKEN FROM: /10-Advanced-Data-Storage-and-Retrieval/2/Activities/01-Ins_Basic_Querying/Solved/Ins_Basic_Querying.ipynb

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float

class BaseballPlayer(Base):
  __tablename__ = "player"
  player_id = Column(String, primary_key=True)
  birth_year = Column(Integer)
  birth_month = Column(Integer)
  birth_day = Column(Integer)
  birth_country = Column(String)
  birth_state = Column(String)
  birth_city = Column(String)
  name_first = Column(String)
  name_last = Column(String)
  name_given = Column(String)
  weight = Column(Integer)
  height = Column(Integer)
  bats = Column(String)
  throws = Column(String)
  debut = Column(String)
  final_game = Column(String)

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Create Database Connection
engine = create_engine('sqlite:///../Resources/database.sqlite')
Base.metadata.create_all(engine)

from sqlalchemy.orm import Session
session = Session(bind=engine)

# Print all of the player names in the database
players = session.query(BaseballPlayer)
for player in players:
  print(player.name_given)

# Find the number of players from the USA
usa = session.query(BaseballPlayer).\
    filter(BaseballPlayer.birth_country == 'USA').count()
print("There are {} players from the USA".format(usa))

# Find those players who were born before 1990
born_before_1990 = session.query(BaseballPlayer).\
    filter(BaseballPlayer.birth_year < 1990).count()
    
print("{} players were born before 1990".format(born_before_1990))

# Find those players from the USA who were born after 1989
born_after_1989 = session.query(BaseballPlayer).\
    filter(BaseballPlayer.birth_year > 1989).filter(BaseballPlayer.birth_country == "USA").\
    count()
print("{} USA players were born after 1989".format(born_after_1989))

In [ ]:
# ----------------------------------------------------------------------------------------
# Reading data using reflection
# TAKEN FROM: /10-Advanced-Data-Storage-and-Retrieval/2/Activities/06-Stu_ReflectingOnSQL/Solved/Stu_Reflection.ipynb

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

# Create engine using the `demographics.sqlite` database file
engine = create_engine("sqlite:///../Resources/demographics.sqlite")

# Declare a Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

# Print all of the classes mapped to the Base
Base.classes.keys()

# Assign the demographics class to a variable called `Demographics`
Demographics = Base.classes.demographics

# Create a session
session = Session(engine)

# Use the session to query Demographics table and display the first 5 locations
for row in session.query(Demographics, Demographics.location).limit(5).all():
    print(row)

# BONUS: Query and print the number of unique Locations
# Hints: Look into counting and grouping operations in SQLAlchemy
locations = session.query(Demographics).group_by(Demographics.location).count()
print(locations)

In [ ]:
# ----------------------------------------------------------------------------------------
# how to get table colums
# TAKEN FROM: /10-Advanced-Data-Storage-and-Retrieval/2/Activities/09-Par_EmojiPlotting/Solved/Stu_Plotting.ipynb

import pandas as pd

# Import SQLAlchemy `automap` and other dependencies here
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

# Create an engine for the `emoji.sqlite` database
engine = create_engine("sqlite:///../Resources/emoji.sqlite", echo=False)

# Use the Inspector to explore the database and print the table names
inspector = inspect(engine)
inspector.get_table_names()

# Use Inspector to print the column names and types
columns = inspector.get_columns('emoji')
for c in columns:
    print(c['name'], c["type"])
    
    # Use `engine.execute` to select and display the first 10 rows from the emoji table
engine.execute('SELECT * FROM emoji LIMIT 10').fetchall()